In [90]:
import torch

# 1. Linear Layer(FC layer) 함수로 만들기

$$\begin{gathered}
y=x\cdot{W}+b, \\
\text{where }x\in\mathbb{R}^{N\times{n}}\text{, }y\in\mathbb{R}^{N\times{m}}. \\
\\
\text{Thus, }W\in\mathbb{R}^{n\times{m}}\text{ and }b\in\mathbb{R}^m.
\end{gathered}$$

In [100]:
# W는 n x m 벡터
W = torch.FloatTensor([[1, 2],   # 3x2  -> |X| = (N, 3),  |y|=(N,2)
                       [3, 4],   # input dimension = 3
                       [5, 6]])  # output dimension = 2

b = torch.FloatTensor([2, 2])    # Nx2

print(W)
print(b)

tensor([[1., 2.],
        [3., 4.],
        [5., 6.]])
tensor([2., 2.])


In [101]:
print(W.size())
print(b.size())

torch.Size([3, 2])
torch.Size([2])


In [102]:
# linear layer 함수
def linear(x, W, b):
    y = torch.matmul(x, W) + b
    
    return y

In [103]:
# 입력값 x 선언
x = torch.FloatTensor([[1, 1, 1],   # 4x3
                       [2, 2, 2],   # N=4
                       [3, 3, 3],
                       [4, 4, 4]])
print(x.size())

torch.Size([4, 3])


In [105]:
# 실제 y를 구해보자
y = linear(x, W, b)

print(y)
y.size()

tensor([[11., 14.],
        [20., 26.],
        [29., 38.],
        [38., 50.]])


torch.Size([4, 2])

# 2. nn.Module 클래스 상속받아서 Linear layer 클래스 만들기

## (low level) nn.Module 상속 받아서 linear layer를 구현

In [34]:
# 파라미터 W, b는 랜덤으로 생성된다.
W = torch.FloatTensor(3, 2)
b = torch.FloatTensor(2)

print(W)
print(b)

tensor([[1.1210e-44, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00]])
tensor([2.0000e+00, 7.3049e+03])


In [56]:
import torch.nn as nn

# nn.Module 클래스를 상속받아서 Linear Layer(FC layer)을 수행하는 클래스 정의
class MyLinear(nn.Module):  # nn.Module 상속
    
    def __init__(self, input_dim=3, output_dim=2):   # 객체 생성 시 input_dim, output_dim 정의
        self.input_dim = input_dim
        self.output_dim = output_dim
        
        super().__init__()  # 상위 클래스의 init 메소드 호출
        
        # nn.Parameter : W, b가 nn.Module을 상속받은 객체의 파라미터라고 선언해준다.        
        self.W = nn.Parameter(torch.FloatTensor(input_dim, output_dim))
        self.b = nn.Parameter(torch.FloatTensor(output_dim))
        
    def forward(self, x):    # x : 입력 데이터  -> |x| = (batch_size, input_dim)
        y = torch.matmul(x, self.W) + self.b
        # |y| = (batch_size, input_dim) * (input_dim, output_dim) = (batch_size, output_dim)        
        return y

Reference: https://pytorch.org/docs/stable/nn.html#torch.nn.Parameter

A kind of Tensor that is to be considered a module parameter.

Parameters are Tensor subclasses, that have a very special property when used with Module s - when they’re assigned as Module attributes they are automatically added to the list of its parameters, and will appear e.g. in parameters() iterator. Assigning a Tensor doesn’t have such effect. This is because one might want to cache some temporary state, like last hidden state of the RNN, in the model. If there was no such class as Parameter, these temporaries would get registered too.

In [64]:
linear = MyLinear(3, 2)  

y = linear(x)

In [65]:
print(y.size())
y

torch.Size([4, 2])


tensor([[1.0400e-32, 2.1684e-19],
        [1.0400e-32, 4.3368e-19],
        [1.0400e-32, 6.5052e-19],
        [1.0400e-32, 8.6736e-19]], grad_fn=<AddBackward0>)

In [66]:
# MyLinear 클래스 내에서 torch.FloatTensor에 의해 랜덤으로 생성된 파라미터 W, b 확인
for p in linear.parameters():
    print(p)

Parameter containing:
tensor([[0.0000e+00, 1.0842e-19],
        [0.0000e+00, 1.0842e-19],
        [9.8091e-45, 0.0000e+00]], requires_grad=True)
Parameter containing:
tensor([1.0400e-32, 1.4013e-45], requires_grad=True)


## (high level) nn.Linear 클래스 사용해서 간편하게 구현

In [76]:
# nn.Linear(input_dim, output_dim) 클래스로 객체(linear)를 생성하면,
# 생성된 linear 객체에 자동으로 forward 함수가 매핑되어 있다(객체 선언 시 자동으로 실행된다.)
# 인자로 forward 함수의 argument(입력 데이터)만 넣어주면 된다.
linear = nn.Linear(3, 2)
y = linear(x)

In [77]:
print(y.size())
y

torch.Size([4, 2])


tensor([[-0.5792, -0.7592],
        [-1.4116, -0.9707],
        [-2.2440, -1.1821],
        [-3.0765, -1.3935]], grad_fn=<AddmmBackward>)

In [78]:
# 랜덤 생성된 파라미터들 확인
for p in linear.parameters():
    print(p)

Parameter containing:
tensor([[-0.4734, -0.2902, -0.0688],
        [-0.1185,  0.2055, -0.2984]], requires_grad=True)
Parameter containing:
tensor([ 0.2533, -0.5478], requires_grad=True)


## nn.Module can contain other nn.Module's child classes.

In [86]:
class MyLinear(nn.Module):

    def __init__(self, input_dim=3, output_dim=2):
        self.input_dim = input_dim
        self.output_dim = output_dim
        
        super().__init__()
        
        self.linear = nn.Linear(input_dim, output_dim)
        
    def forward(self, x):
        # |x| = (batch_size, input_dim)
        y = self.linear(x)
        # |y| = (batch_size, output_dim)
        
        return y

In [87]:
# Linear layer 출력값 생성
linear = MyLinear(3, 2)
y = linear(x)

In [88]:
print(y.size())
y

torch.Size([4, 2])


tensor([[-1.0333,  0.3377],
        [-1.8432,  1.0565],
        [-2.6531,  1.7753],
        [-3.4630,  2.4941]], grad_fn=<AddmmBackward>)

In [89]:
# 랜덤 생성된 파라미터들 확인
for p in linear.parameters():
    print(p)

Parameter containing:
tensor([[-0.1130, -0.1795, -0.5174],
        [ 0.2900, -0.0122,  0.4410]], requires_grad=True)
Parameter containing:
tensor([-0.2234, -0.3811], requires_grad=True)
